In [1]:
import socket
import threading
import csv
import os
import random

# Define constants
SERVER_IP = "127.0.0.1"
SERVER_PORT = 4000
CSV_FILE = "users.csv"
BOOKINGS_FILE = "bookings.csv"
TRAINS_FILE = "trains.csv"
MAX_SEATS = 100

# FCFS (First-Come-First-Serve) scheduling algorithm for ticket booking
def fcfs_schedule(train_id, seats, prn):
    with open(BOOKINGS_FILE, mode='r+') as file:
        reader = csv.DictReader(file)
        bookings = list(reader)
        updated = False
        for booking in bookings:
            if booking['Train ID'] == train_id:
                if int(booking['Seats']) + int(seats) <= MAX_SEATS:
                    booking['Seats'] = str(int(booking['Seats']) + int(seats))
                    updated = True
                    break
        if not updated:
            bookings.append({'PRN': prn, 'Train ID': train_id, 'Seats': seats})
        file.seek(0)
        writer = csv.DictWriter(file, fieldnames=['PRN', 'Train ID', 'Seats'])
        writer.writeheader()
        for booking in bookings:
            writer.writerow(booking)

def fcfs_cancel(train_id, seats):
    with open(BOOKINGS_FILE, mode='r+') as file:
        reader = csv.DictReader(file)
        bookings = list(reader)
        for booking in bookings:
            if booking['Train ID'] == train_id:
                booking['Seats'] = str(int(booking['Seats']) - int(seats))
                break
        file.seek(0)
        writer = csv.DictWriter(file, fieldnames=['PRN', 'Train ID', 'Seats'])
        writer.writeheader()
        for booking in bookings:
            writer.writerow(booking)

def generate_prn():
    return random.randint(1000000000, 9999999999)

def handle_client(client_socket, client_address):
    print(f"Connection from {client_address} established.")
    while True:
        try:
            choice = int(client_socket.recv(1024).decode())
            if choice == 1:
                if handle_login(client_socket):
                    continue
            elif choice == 2:
                if handle_registration(client_socket):
                    continue
            elif choice == 3:
                print(f"Client {client_address} disconnected.")
                break
            elif choice == 4:
                handle_book_ticket(client_socket)
            elif choice == 5:
                handle_cancel_ticket(client_socket)
            elif choice == 6:
                handle_train_status(client_socket)
            else:
                client_socket.sendall(b"Invalid choice. Please try again.")
        except Exception as e:
            print(f"Error handling client {client_address}: {e}")
            break
    client_socket.close()
    print(f"Connection with {client_address} closed.")

def handle_login(client_socket):
    data = client_socket.recv(1024).decode().strip()
    name, code = data.split(',')
    with open(CSV_FILE, mode='r') as file:
        reader = csv.reader(file)
        users = {rows[0]: (rows[1], rows[2], rows[3]) for rows in reader}
    if name in users and users[name][0] == code:
        client_socket.sendall(b"1")  # Login successful
        print(f"Login successful for {name}.")
        return True
    else:
        client_socket.sendall(b"0")  # Login failed
        print(f"Login failed for {name}.")
        return False

def handle_registration(client_socket):
    registration_details = client_socket.recv(1024).decode().strip()
    name, code, address, phone = registration_details.split(',')
    with open(CCSV_FILE, mode='a', newline='') as file:
        fieldnames = ['Name', 'Code', 'Address', 'Phone']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writerow({'Name': name, 'Code': code, 'Address': address, 'Phone': phone})
    client_socket.sendall(b"1")  # Registration successful
    print(f"Registration successful for {name}.")
    return True

def handle_book_ticket(client_socket):
    train_id = client_socket.recv(1024).decode().strip()
    seats = client_socket.recv(1024).decode().strip()
    prn = generate_prn()
    fcfs_schedule(train_id, seats, prn)
    with open(TRAINS_FILE, mode='r+') as file:
        reader = csv.DictReader(file)
        trains = list(reader)
        updated_trains = []
        for train in trains:
            if train['Train ID'] == train_id:
                updated_seats = int(train['Available Seats']) - int(seats)
                updated_trains.append({'Train ID': train_id, 'Available Seats': str(updated_seats)})
            else:
                updated_trains.append(train)
        file.seek(0)
        writer = csv.DictWriter(file, fieldnames=['Train ID', 'Available Seats'])
        writer.writeheader()
        for train in updated_trains:
            writer.writerow(train)
    ticket_details = f"PRN: {prn}, Train ID: {train_id}, Seats: {seats}"
    client_socket.sendall(ticket_details.encode())  # Send ticket details to client
    print(f"Booking of {seats} seats for train {train_id} successful with PRN: {prn}.")

def handle_cancel_ticket(client_socket):
    prn = client_socket.recv(1024).decode().strip()
    with open(BOOKINGS_FILE, mode='r+') as file:
        reader = csv.DictReader(file)
        bookings = list(reader)
        for booking in bookings:
            if booking['PRN'] == prn:
                train_id = booking['Train ID']
                seats = booking['Seats']
                bookings.remove(booking)
                fcfs_cancel(train_id, seats)
                break
        else:
            client_socket.sendall(b"0")  # Ticket cancellation failed
            print(f"Ticket cancellation failed for PRN: {prn}.")
            return
        file.seek(0)
        writer = csv.DictWriter(file, fieldnames=['PRN', 'Train ID', 'Seats'])
        writer.writeheader()
        for booking in bookings:
            writer.writerow(booking)
    with open(TRAINS_FILE, mode='r+') as file:
        reader = csv.DictReader(file)
        trains = list(reader)
        for train in trains:
            if train['Train ID'] == train_id:
                available_seats = int(train['Available Seats']) + int(seats)
                train['Available Seats'] = str(available_seats)
                break
        file.seek(0)
        writer = csv.DictWriter(file, fieldnames=['Train ID', 'Available Seats'])
        writer.writeheader()
        for train in trains:
            writer.writerow(train)
    client_socket.sendall(b"1")  # Ticket cancellation successful
    print(f"Ticket cancellation successful for PRN: {prn}.")

def handle_train_status(client_socket):
    train_id = client_socket.recv(1024).decode().strip()
    with open(BOOKINGS_FILE, mode='r') as file:
        reader = csv.DictReader(file)
        bookings = list(reader)
        total_seats_booked = sum(int(booking['Seats']) for booking in bookings if booking['Train ID'] == train_id)
        available_seats = MAX_SEATS - total_seats_booked
        status = f"Train ID: {train_id}, Booked Seats: {total_seats_booked}, Available Seats: {available_seats}"
        client_socket.sendall(status.encode())
        print(f"Train status sent for Train ID: {train_id}.")

def main():
    if not os.path.exists(CSV_FILE):
        with open(CSV_FILE, 'w', newline='') as file:
            fieldnames = ['Name', 'Code', 'Address', 'Phone']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()

    if not os.path.exists(BOOKINGS_FILE):
        with open(BOOKINGS_FILE, 'w', newline='') as file:
            fieldnames = ['PRN', 'Train ID', 'Seats']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()

    if not os.path.exists(TRAINS_FILE):
        with open(TRAINS_FILE, 'w', newline='') as file:
            fieldnames = ['Train ID', 'Available Seats']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
        # Adding sample train data
        trains_data = [
            {'Train ID': '101', 'Available Seats': '50'},
            {'Train ID': '102', 'Available Seats': '60'},
            {'Train ID': '103', 'Available Seats': '70'},
            {'Train ID': '104', 'Available Seats': '80'},
            {'Train ID': '105', 'Available Seats': '90'}
        ]
        with open(TRAINS_FILE, 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            for train in trains_data:
                writer.writerow(train)

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        server_socket.bind((SERVER_IP, SERVER_PORT))
        server_socket.listen(5)
        print(f"Server listening on {SERVER_IP}:{SERVER_PORT}")
        while True:
            client_socket, client_address = server_socket.accept()
            # Create a new thread to handle the client
            client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
            client_thread.start()
    except Exception as e:
        print(f"Error in the server: {e}")
    finally:
        server_socket.close()

if __name__ == "__main__":
    main()


Server listening on 127.0.0.1:4000
